# Multi-Channel TLS Demo with Toyblocks Scene
Notebook: Hannah Weiser & Sina Zumstein, 2023

This demo scene showcases toyblock models scanned by a terrestial laserscanner. We will use the command-line access of HELIOS++ to run the simulation, and use Python just for displaying the input XMLs and the resulting point clouds.



In [ ]:
import sys, os
from pathlib import Path
from IPython.display import Code

current_folder = globals()["_dh"][0]
helios_path = str(Path(current_folder).parent)
sys.path.append(helios_path)  # add helios-plusplus directory to PATH
import pyhelios

from pyhelios.util.xmldisplayer import display_xml, find_playback_dir

## Survey
Let us look at the XML files in the simulation. First, we investigate the survey XML file, `livox_test.xml`:

In [ ]:
os.chdir(helios_path)
Code(display_xml(r'data\surveys\toyblocks\livox_test.xml'), language='XML')

We can see that there are two `leg` elements corresponding to the two scan positions (SPs) at `(-10.01, -15.01)` and `(-10, -15.0)`. This seems strange?

## Platform 


## Scanner
Now let's look at the scanner that is placed on this platform, the `livox_mid-70` defined in `data\scanners_tls.xml` (as shown in the survey XML):

In [ ]:
Code(display_xml(r'data/scanners_als.xml', 'livox-mid-100'), language='XML')

Besides the typical properties, such as `beamDivergence_rad` or `accuracy`, we see a special feature: The scanner has three channels with a different `beamOrigin` each. This way, the multi-channel laser scanner Livox Mid-100 is simulated, which has three LiDAR sensors with overlapping fields of view.

<center><img src=".\images\Livox_img.png" alt="Livox" title="Livox technical figure" width="500">

<i>Multi-channel laser scanner Livox Mid-100. Source: [Livox datasheet](https://www.livoxtech.com/3296f540ecf5458a8829e01cf429798e/downloads/20190530/Livox%20Mid-100%20Quick%20Start%20Guide%20multi%20v1.4.pdf).</i></center>

## Scene

Finally, let us take a look at the scene, `toyblocks_scene` in `data/scenes/toyblocks/toyblocks_scene.xml`:

In [ ]:
Code(display_xml(r'data/scenes/toyblocks/toyblocks_scene.xml', 'toyblocks_scene'))

This is a simple scene, consisting of a groundplane, two cubes, a sphere and a cylinder. The objects are loaded with the `objloader` filter. The cubes are loaded from the same file (`data/sceneparts/cube.obj`) but the second one is additionally rotated, scaled and translated. The sphere is just scaled while the cylinder is not modified at all.

## Executing the Simulation

Next, we will run the simulation. In Jupyter Notebooks, we can run external commands with the `!command` syntax, but you can also just run it from the command line. In this demo we will run the simulation twice: One time with all three scanners working together, producing one output per strip and one time with the `--splitByChannel` argument, which will create individual outputs for each scanner. We start with the joined simulation.

In [ ]:
!"run/helios.exe" data\surveys\toyblocks\livox_test.xml
output_path_joined = find_playback_dir(r"data\surveys\toyblocks\livox_test.xml")

In [ ]:
!"run/helios.exe" data\surveys\toyblocks\livox_test.xml --splitByChannel
output_path_split_by_channel = find_playback_dir(r"data\surveys\toyblocks\livox_test.xml")

## The results 
Now we can display the 3D plots. 

### 1) Full point cloud
We start with the point cloud of the first simuation and we color it by **object ID**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from pathlib import Path

print("Loading points from", Path(output_path_joined).relative_to(helios_path).as_posix())


SP_1 = np.loadtxt(Path(output_path_joined) / 'leg000_points.xyz')
SP_2 = np.loadtxt(Path(output_path_joined) / 'leg001_points.xyz')

#stacking the SPs 
SPs= np.vstack((SP_1, SP_2))

traj_1 = np.loadtxt(Path(output_path_joined) / 'leg000_trajectory.txt')
traj_2 = np.loadtxt(Path(output_path_joined) / 'leg001_trajectory.txt')

traj = np.vstack((traj_1[:, :3], traj_2[:, :3]))

In [ ]:
def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius]) 
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [ ]:
# Matplotlib figures.
fig = plt.figure(figsize=(15,10))


#settings for a discrete colorbar
N=5
cmap=plt.get_cmap('jet',N)

# Scatter plot of first scanner (coloured by hitObjectId).
ax = fig.add_subplot(1,3,1, projection='3d')
sc = ax.scatter(SPs[:, 0], Sps[:, 1], SPs[:, 2], c=SPs[:, 8], cmap=cmap, s=0.02, label='scene')

# Plot of trajectory.
ax.plot(traj[:,0], traj[:,1], traj[:,2], c = 'black', label = 'scanner trajectory')

# Add axis labels.
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.set_zlabel('$Z$')

set_axes_equal(ax)    

# Set title.
ax.set_title(label='Point cloud and trajectory',fontsize=18)

cax = plt.axes([0.4, 0.35, 0.01, 0.25])
cbar = plt.colorbar(sc, cax=cax, ticks=[2/3,4/3,2,8/3,10/3])

cbar.set_label("Object Id", fontsize=15)
cbar.ax.set_yticklabels(['0', '1', '2', '3','4'])

# Display results
plt.show()
